### Data Loading

In [93]:
from langchain_community.document_loaders import TextLoader
file_path = "/Users/toshalimohapatra/Documents/Chatbot_Langchain/data/textfile.txt"
loader = TextLoader(file_path)
text_documents = loader.load()
text_documents[:1]

[Document(metadata={'source': '/Users/toshalimohapatra/Documents/Chatbot_Langchain/data/textfile.txt'}, page_content='Brains are highly fashionable. A very common belief is that the human mind and cognition lie in the head, within the confines of a spongy greyish organ called ‘the brain’ encased within skull and skin. While I was looking at that Thinker’s head, with my stomach protesting, I realised I had completely dismissed his body. I was obsessed with understanding his mind, and my mind via his brain, but I completely ignored the rest of him. Like the great majority of scientists and philosophers throughout history, I too was putting cognition into the head and forgetting about the body, treating it as something that got in the way of pure cerebral thought.\n\nBeing a philosopher hadn’t been my only dream.\n\nWhy are we so reluctant to consider the brain as just another part of the body?\n\nI’d decided to study philosophy after becoming a frustrated musician. My parents, God bless 

In [94]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [95]:
from langchain_community.llms import Ollama
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv('LANGCHAIN_TRACING_V2', 'true')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY', '')

In [96]:
## Read from web based loader
from langchain_community.document_loaders import WebBaseLoader
from bs4.filter import SoupStrainer

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


In [97]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [98]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=200,  
    add_start_index=True,  
)
all_splits = text_splitter.split_documents(docs)
print(all_splits[:5])
print(f"Split blog post into {len(all_splits)} sub-documents.")

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from

In [99]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "/Users/toshalimohapatra/Documents/Chatbot_Langchain/data/GPT_1.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
docs[0]

Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-06-08T19:14:34+00:00', 'author': '', 'keywords': '', 'moddate': '2025-09-23T22:25:51+05:30', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017) kpathsea version 6.2.3', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/toshalimohapatra/Documents/Chatbot_Langchain/data/GPT_1.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Improving Language Understanding\nby Generative Pre-Training\nAlec Radford\nOpenAI\nalec@openai.com\nKarthik Narasimhan\nOpenAI\nkarthikn@openai.com\nTim Salimans\nOpenAI\ntim@openai.com\nIlya Sutskever\nOpenAI\nilyasu@openai.com\nAbstract\nNatural language understanding comprises a wide range of diverse tasks such\nas textual entailment, question answering, semantic similarity assessment, and\ndocument classiﬁcation. Although large unlabeled text corpora are abundant,\nlabeled data for l

### Create Embeddings

In [100]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model="qwen2.5-coder:0.5b",
)

In [101]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# Show the retrieved document's content
print(retrieved_documents[0].page_content)

LangChain is the framework for building context-aware reasoning applications


In [102]:
single_vector = embeddings.embed_query(text)
print(str(single_vector)[:100])  # Show the first 100 characters of the vector

[0.009889353, 0.011473607, -0.0017168578, -0.018002266, 0.0035788615, -0.0030870365, 0.0033981635, 0


In [103]:
text2 = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs"
)
two_vectors = embeddings.embed_documents([text, text2])
for vector in two_vectors:
    print(str(vector)[:100])  # Show the first 100 characters of the vector

[0.009889353, 0.011473607, -0.0017168578, -0.018002266, 0.0035788615, -0.0030870365, 0.0033981635, 0
[-0.0021199523, -0.0019641852, -0.00074170635, -0.01886839, 0.0031963116, -0.007960364, 0.042872794,


### Vector DB

In [104]:
## ChromaDB implementation 
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(
    documents=docs[:10],
    embedding=embeddings,
)

In [105]:
query = "Where was GLUE benchmark used?"
result = vector_store.similarity_search(query, k=2)
result[1].page_content

'Improving Language Understanding\nby Generative Pre-Training\nAlec Radford\nOpenAI\nalec@openai.com\nKarthik Narasimhan\nOpenAI\nkarthikn@openai.com\nTim Salimans\nOpenAI\ntim@openai.com\nIlya Sutskever\nOpenAI\nilyasu@openai.com\nAbstract\nNatural language understanding comprises a wide range of diverse tasks such\nas textual entailment, question answering, semantic similarity assessment, and\ndocument classiﬁcation. Although large unlabeled text corpora are abundant,\nlabeled data for learning these speciﬁc tasks is scarce, making it challenging for\ndiscriminatively trained models to perform adequately. We demonstrate that large\ngains on these tasks can be realized by generative pre-training of a language model\non a diverse corpus of unlabeled text, followed bydiscriminative ﬁne-tuning on each\nspeciﬁc task. In contrast to previous approaches, we make use of task-aware input\ntransformations during ﬁne-tuning to achieve effective transfer while requiring\nminimal changes to the m

In [106]:
## FAISS Implementation
from langchain_community.vectorstores import FAISS  
faiss_vector_store = FAISS.from_documents(
    documents=docs[:10],
    embedding=embeddings,
)

In [107]:
query = "Where was GLUE benchmark used?"
result = vector_store.similarity_search(query, k=2)
result[1].page_content

'Improving Language Understanding\nby Generative Pre-Training\nAlec Radford\nOpenAI\nalec@openai.com\nKarthik Narasimhan\nOpenAI\nkarthikn@openai.com\nTim Salimans\nOpenAI\ntim@openai.com\nIlya Sutskever\nOpenAI\nilyasu@openai.com\nAbstract\nNatural language understanding comprises a wide range of diverse tasks such\nas textual entailment, question answering, semantic similarity assessment, and\ndocument classiﬁcation. Although large unlabeled text corpora are abundant,\nlabeled data for learning these speciﬁc tasks is scarce, making it challenging for\ndiscriminatively trained models to perform adequately. We demonstrate that large\ngains on these tasks can be realized by generative pre-training of a language model\non a diverse corpus of unlabeled text, followed bydiscriminative ﬁne-tuning on each\nspeciﬁc task. In contrast to previous approaches, we make use of task-aware input\ntransformations during ﬁne-tuning to achieve effective transfer while requiring\nminimal changes to the m

### Chains and Retriever

In [122]:
## Chat prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the context provided. Think logically step by steo before providing a detailed answer. Be respectful, detailed, and professional in your response.
<context>{context}</context>
Question: {input}
""")

In [125]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

llm = Ollama(model="qwen2.5-coder:0.5b")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,      # whatever PromptTemplate you defined
)

retriever = faiss_vector_store.as_retriever()

retrieval_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=document_chain,
)

response = retrieval_chain.invoke({"input": "Where was GLUE benchmark used?"})

In [126]:
response["answer"]

'GLUE benchmark was used to evaluate the performance of the model in 9 out of the 12 datasets we evaluated on.'